Collecting and Storing Data from the Web
===
**Using BeautifulSoup, Pandas, and SQL**

*by Ryan Delgado*


Data collection is the first step in the data science lifecycle. Ideally, the data is easily-available, properly-formatted, and free of any data quality issues. More often than not, however, this isn't the case. The data we need may be on the web but inaccessible through an API and can contain missing values or incorrect data. This tutorial will show how Python tools can be used to extract and process data from the web, and then warehouse it in a relational database. The purpose is to simulate a a data collection process that facilitates data analysis.  The tutorial is divided into three main sections:
1. Introduction to the data and tools
2. Basic extraction and processing of data
3. Storing data in a relational database

The tutorial will conclude with potential future directions. 

Data and Tools
===

We'll work with Short Interest data for US Equities. Short Interest is the quantity of shares that investors are holding short. This figure (and the Short Interest Ratio) are useful for guaging market sentiment. Short Interest data isn't easily available on the web, but it can be bought (e.g. from http://shortsqueeze.com), or it can be found on www.nasdaq.com:

<img src='goog_screenshot.png'>

We could copy and paste this data into Excel worksheets, but this is very tedious for a large number of stocks, and we'd need to repeat this every 2 weeks to keep up with the data. It's easier to instead automate this with several tools:
+ **requests** - A standard Python library for downloading web pages. We'll use this to download the HTML pages into memory for parsing.
+ **BeautifulSoup4** - A screen scraping library that we'll use to parse HTML pages
+ **Pandas/NumPy** - Standard Python data analysis and manipulation libraries we'll use to process the data.
+ **SQLite3** - A Python API to interact with SQLite databases, which are simple and small relational databases. We'll use this library to warehouse the data in a simple relational database.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Basic Data Extraction and Processing
===

We'll start by figuring out the list of stocks we want to gather data on. We'll focus on the 20 most actively traded equities in Nasdaq in this tutorial. A table containing these stocks can be found at http://www.nasdaq.com/markets/most-active.aspx:

<img src='most_active.PNG'>

We'll download and parse the page, and then extract the table of interest into a pandas DataFrame.

In [2]:
request = requests.get('http://www.nasdaq.com/markets/most-active.aspx')
content = request.content
mostactive_parser = BeautifulSoup(content, 'html.parser')
mostactive_table = str(mostactive_parser.select('#_active > table')[0])

Notes:
+ '#_active > table' is a **CSS Selector*** that I found using Chrome Dev Tools. This selector identifies the element of itnerest on a webpage.
+ BeautifulSoup's select function returns a list of elements that match the select statement. In this case it returns a single-element list, and we access that element with the [0] index. We also convert it to a string so it can be read by pandas.

In [3]:
mostactive_table_pd = pd.read_html(mostactive_table)[0]
assert mostactive_table_pd.columns.tolist() == ['Symbol', 'Company', 'Last Sale*', 'Change Net / %', 'Share Volume', 'Unnamed: 5']
assert len(mostactive_table_pd) == 20

mostactive_table_pd.head()

,Symbol,Company,Last Sale*,Change Net / %,Share Volume,Unnamed: 5
0,SIRI,Sirius XM Holdings Inc.,NaN,$ 4.11,unch,75359436
1,AAPL,Apple Inc.,NaN,$ 113.72,0.76 ▼ 0.66%,34778615
2,XIV,VelocityShares Daily Inverse VIX Short Term ETN,NaN,$ 37.65,1.70 ▼ 4.32%,34648336
3,QQQ,"PowerShares QQQ Trust, Series 1",NaN,$ 117.10,0.74 ▼ 0.63%,32113033
4,MSFT,Microsoft Corporation,NaN,$ 59.87,0.23 ▼ 0.38%,31551070


Great. We have the table of most active stocks in a DataFrame. Let's convert the Symbol column to a list so we can iterate through it.

In [4]:
mostactive_tickers = mostactive_table_pd['Symbol'].tolist()
print(mostactive_tickers)

['SIRI', 'AAPL', 'XIV', 'QQQ', 'MSFT', 'FB', 'MU', 'FTR', 'INTC', 'CSCO', 'AMGN', 'QCOM', 'GRPN', 'TVIX', 'CMCSA', 'NXPI', 'ZNGA', 'GILD', 'GT', 'PYPL']


We're ready to download and process the short interest data for each stock. We'll account for common issues with web scraping, such as:
+ Missing data, as denoted by "-", "N/A" or some other string. We can detect this by checking the dtype of the columns and ensuring the columns we expect to be numeric are indeed numeric (e.g. int64 or float64).
+ The table simply not showing up on the page. This would happen if we try to scrape a stock from Nasdaq.com that doesn't trade on Nasdaq.

We'll iterate through each stock in the list, and perform these steps:
1. Download the web page for the stock, and check that there is indeed short interest data
2. Parse the HTML using BeautifulSoup, and extract the short interest data into a DataFrame
3. Check the quality of the data - ensure the columns we expect to be in the table are there, and the dtypes are what we expect them to be.
4. If the checks pass, append the DataFrame to our list of DataFrames


In [5]:
nasdaq_frames = []
    
for ticker in mostactive_tickers:
    
    # Download HTML page for ticker's short interest
    print('Scraping Nasdaq short interest data for {}'.format(ticker))
    req = requests.get('http://www.nasdaq.com/symbol/{}/short-interest'.format(ticker))
    content = str(req.content)

    # Stocks that don't have short interest data on Nasdaq will show a page that says 
    # "This page does not support NYSE and AMEX Short Interest." We can search for this string in the page to use as an 
    # indicator that there isn't short interest data for this stock. If we find this string in the page, we'll skip this
    # iteration and move on to the next ticker.
    if 'This page does not support' in content:
        print('Short Interest data is not available for {}'.format(ticker))
        continue

    # Parse the HTML page, then capture the Short Interest table using its CSS Selector (found using Chrome dev tools)
    nasdaq_parser = BeautifulSoup(content, 'html.parser')    
    nasdaq_table = str(nasdaq_parser.select('#quotes_content_left_ShortInterest1_ShortInterestGrid')[0])
    nasdaq_frame = pd.read_html(nasdaq_table)[0]

    # Quality checks. We know previously that there are always four columns. Check that the four column names 
    # match what we expect and that the data types of the columns match what we expect. If the
    # checks do not pass, print a message then skip to the next iteration.
    try:
        assert nasdaq_frame.columns.tolist() == ['Settlement Date', 'Short Interest', 'Avg Daily Share Volume', 'Days To Cover']
        assert nasdaq_frame.dtypes.tolist() == [np.dtype('O'), np.dtype('int64'), np.dtype('int64'), np.dtype('float64')]        
    except AssertionError:
        print('Quaity checks failed for {}. This data will not be appended to Nasdaq frames list.'.format(ticker))
        continue

    # If the quality checks pass, add a column for the symbol and append to the nasdaq_frames list.
    print('Quality checks passed for {}. Appending to Nasdaq frames list'.format(ticker))
    nasdaq_frame['symbol'] = ticker
    nasdaq_frames.append(nasdaq_frame)

# Summary report
total_successes = len(nasdaq_frames)
total_tickers = len(mostactive_tickers)
print('Short Interest data scraped successfully for {} out of {} stocks'.format(total_successes, total_tickers))

Scraping Nasdaq short interest data for SIRI
Quality checks passed for SIRI. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for AAPL
Quality checks passed for AAPL. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for XIV
Quality checks passed for XIV. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for QQQ
Quality checks passed for QQQ. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for MSFT
Quality checks passed for MSFT. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for FB
Quality checks passed for FB. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for MU
Quality checks passed for MU. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for FTR
Quality checks passed for FTR. Appending to Nasdaq frames list
Scraping Nasdaq short interest data for INTC
Quality checks passed for INTC. Appending to Nasdaq frames list
Scraping Nasdaq short interest da

Let's look at the frame for Sirius XM as an example.

In [6]:
siri_frame = nasdaq_frames[0]
siri_frame.head()

,Settlement Date,Short Interest,Avg Daily Share Volume,Days To Cover,symbol
0,10/14/2016,207206065,40275240,5.144751,SIRI
1,9/30/2016,220922513,44733332,4.938655,SIRI
2,9/15/2016,229413860,55949317,4.100387,SIRI
3,8/31/2016,221459290,53000940,4.178403,SIRI
4,8/15/2016,221124840,51604416,4.284998,SIRI


Storing the Data in a Relational Database
===

We're ready to store the data. We can easily store these tables as separate CSV files in a local file system, but a more elegant and space-efficient solution is to store the data in a relational database. Let's create a basic SQLite database with 5 tables:
+ A *Company* dimension table, containing the symbol and full company name of each stock
+ A *Measure* dimension table, containing the measure names (Short Interest, Days To Cover, etc.)
+ A *Date* dimension table, containing the dates each value could take
+ An *Equity_historical* fact table, containing foreign keys for the dimensions and the actual values for the data we extract
+ An *Equity_stage* table that's an intermediate staging table for adding the data into the database.

In [7]:
import sqlite3

# Create Equities_mart database
conn = sqlite3.connect('equities_mart.db')
cursor = conn.cursor()

# Create Company, Measure, and Date dimension tables
cursor.execute('''
CREATE TABLE Company(
    id     integer PRIMARY KEY,
    symbol text,
    name   text
    )
''')

cursor.execute('''
CREATE TABLE Measure(
    id           integer PRIMARY KEY,
    measure_name text
    )
''')

cursor.execute('''
CREATE TABLE Date(
    id   integer PRIMARY KEY,
    date text
    )
''')

# Create Equity_historical fact table
cursor.execute('''
CREATE TABLE Equity_historical(
    id         integer PRIMARY KEY,
    company_id integer,    
    date_id    integer,
    measure_id integer,
    value      real,
    FOREIGN KEY(company_id) REFERENCES Company(id),
    FOREIGN KEY(measure_id) REFERENCES Measure(id),
    FOREIGN KEY(date_id) REFERENCES Date(id)
    )
''')

# Create Equity_stage table
cursor.execute('''
CREATE TABLE Equity_stage(
    id      integer PRIMARY KEY,
    date    text,    
    symbol  text,
    measure text,
    value   real
    )
''')

Let's populate the dimension tables based on the data we scraped from Nasdaq

In [8]:
company_frame = mostactive_table_pd[['Symbol','Company']]
company_frame.columns = ['symbol','name']
company_frame.to_sql(name='Company', con=conn, if_exists='append', index=False)

measure_frame = pd.DataFrame(['Short Interest', 'Avg Daily Share Volume', 'Days To Cover'])
measure_frame.columns = ['measure_name']
measure_frame.to_sql(name='Measure', con=conn, if_exists='append', index=False)

date_frame = pd.DataFrame(siri_frame['Settlement Date'])
date_frame.columns = ['date']
date_frame.to_sql(name='Date', con=conn, if_exists='append', index=False)

The short interest frames are "wide" tables, in that the measures are individual columns. This is ideal for data analysis, but doesn't quite fit with the database schema. We'll define a function that "elongates" the frame (using the pandas melt function), which we can then write to the stage table.

In [9]:
def prep_stage_frame(raw_frame):

    # Change the dtypes of the ShortInterest and ShareVolume columns to float64 so there aren't any conflicts when melting
    # the frame.
    raw_frame['Short Interest'] = raw_frame['Short Interest'].astype(np.dtype('float64'))
    raw_frame['Avg Daily Share Volume'] = raw_frame['Avg Daily Share Volume'].astype(np.dtype('float64'))
    
    # Melt raw_frame so that the measure columns are stacked into one column, and the measure names are placed in 
    # another column. Keep the Date and Symbol columns the same.
    melted_frame = pd.melt(raw_frame, id_vars=['Settlement Date', 'symbol'], 
                           value_vars=['Short Interest','Avg Daily Share Volume','Days To Cover'])
    
    # Rename columns to fit in the staging table
    melted_frame.columns = ['date','symbol','measure','value']
    
    return melted_frame


Now let's collect the short interest data again, but we'll append the data to the Equity_stage table instead of a list of DataFrames.

In [10]:
for ticker in mostactive_tickers:
    
    # Download HTML page for ticker's short interest
    print('Scraping Nasdaq short interest data for {}'.format(ticker))
    req = requests.get('http://www.nasdaq.com/symbol/{}/short-interest'.format(ticker))
    content = str(req.content)

    # Stocks that don't have short interest data on Nasdaq will show a page that says 
    # "This page does not support NYSE and AMEX Short Interest." We can search for this string in the page to use as an 
    # indicator that there isn't short interest data for this stock. If we find this string in the page, we'll skip this
    # iteration and move on to the next ticker.
    if 'This page does not support' in content:
        print('Short Interest data is not available for {}'.format(ticker))
        continue

    # Parse the HTML page, then capture the Short Interest table using its CSS Selector (found using Chrome dev tools)
    nasdaq_parser = BeautifulSoup(content, 'html.parser')    
    nasdaq_table = str(nasdaq_parser.select('#quotes_content_left_ShortInterest1_ShortInterestGrid')[0])
    nasdaq_frame = pd.read_html(nasdaq_table)[0]

    # Quality checks. We know previously that there are always four columns and 25 rows. Check that the four column names 
    # match what we expect, that the data types of the columns match what we expect, and that there are 25 rows. If the
    # checks do not pass, print a message then
    try:
        assert nasdaq_frame.columns.tolist() == ['Settlement Date', 'Short Interest', 'Avg Daily Share Volume', 'Days To Cover']
        assert nasdaq_frame.dtypes.tolist() == [np.dtype('O'), np.dtype('int64'), np.dtype('int64'), np.dtype('float64')]        
        assert len(nasdaq_frame) == 25
    except AssertionError:
        print('Quaity checks failed for {}. This data will not be appended to Nasdaq frames list.'.format(ticker))
        continue

    # If the quality checks pass, add a column for the symbol and append to the nasdaq_frames list.  
    # Transform the nasdaq_frame into a DataFrame that can fit in the stage table, then append to Equity_stage.
    print('Quality checks passed for {}. Appending to Equity_stage'.format(ticker))
    nasdaq_frame['symbol'] = ticker
    prepped_stage_frame = prep_stage_frame(nasdaq_frame)
    prepped_stage_frame.to_sql(name='Equity_stage', con=conn, if_exists='append', index=False)



Scraping Nasdaq short interest data for SIRI
Quality checks passed for SIRI. Appending to Equity_stage
Scraping Nasdaq short interest data for AAPL
Quality checks passed for AAPL. Appending to Equity_stage
Scraping Nasdaq short interest data for XIV
Quality checks passed for XIV. Appending to Equity_stage
Scraping Nasdaq short interest data for QQQ
Quality checks passed for QQQ. Appending to Equity_stage
Scraping Nasdaq short interest data for MSFT
Quality checks passed for MSFT. Appending to Equity_stage
Scraping Nasdaq short interest data for FB
Quality checks passed for FB. Appending to Equity_stage
Scraping Nasdaq short interest data for MU
Quality checks passed for MU. Appending to Equity_stage
Scraping Nasdaq short interest data for FTR
Quality checks passed for FTR. Appending to Equity_stage
Scraping Nasdaq short interest data for INTC
Quality checks passed for INTC. Appending to Equity_stage
Scraping Nasdaq short interest data for CSCO
Quality checks passed for CSCO. Appending 

We'll normalize the data in the Equity_stage table based on the dimension tables, and insert this normalized table into the Equity_historical table. We'll empty the Equity_stage table after the data has been successfully transferred.

In [11]:
# Set stage table to the fact table
cursor.execute('''
INSERT INTO Equity_historical (company_id, date_id, measure_id, value)
SELECT c.id as company_id, d.id as date_id, m.id as measure_id, value
    FROM Equity_stage e
    JOIN Company c
        ON e.symbol = c.symbol
    JOIN Date d
        ON e.date = d.date
    JOIN Measure m
        ON e.measure = m.measure_name
    ;
'''
)

# Clear contents of the stage table 
cursor.execute('''
DELETE FROM Equity_stage;
''')

Let's do a quick query on the Equity_historical table to make sure the data looks as expected.

In [12]:
query = '''
SELECT * FROM Equity_historical;
'''

historical_frame = pd.read_sql(sql=query, con=conn)
historical_frame.head()

,id,company_id,date_id,measure_id,value
0,1,1,1,1,207206065.0
1,2,1,2,1,220922513.0
2,3,1,3,1,229413860.0
3,4,1,4,1,221459290.0
4,5,1,5,1,221124840.0


Excellent. We can now query the database to retrieve data we want. As an example, let's query Date & Short Interest data for AAPL.

In [13]:
query = '''
SELECT d.date as Date, e.value as Short_Interest
    FROM Equity_historical e
    JOIN Date d
        ON e.date_id = d.id
    JOIN Company c
        ON e.company_id = c.id
    JOIN Measure m
        ON e.measure_id = m.id
    WHERE c.symbol       = "AAPL"
    AND   m.measure_name = "Short Interest"
    ;
'''

aapl_shortinterest_historical = pd.read_sql(query, conn)
aapl_shortinterest_historical.head()

,Date,Short_Interest
0,10/14/2016,56005114.0
1,9/30/2016,54716610.0
2,9/15/2016,60135782.0
3,8/31/2016,47083129.0
4,8/15/2016,51758571.0


Voila! We've successfully extracted the data from Nasdaq.com and efficiently stored it in a relational database. The data is ready for analysis.

Conclusion
===

This tutorial showed an example ETL workflow for collecting and storing data from the web using BeautifulSoup, pandas, and SQLite. This is only an example workflow that could be used for small, individual projects. We could improve this by
+ Using a more scalable RDBMS like MySQL or Oracle instead of SQLite, perhaps in a cloud-based solution like Amazon Redshift.
+ Using Scrapy instead of requests/BeautifulSoup. Scrapy is a web scraping framework rather than just a parsing library like BeautifulSoup or LXML, and it has more capabilities.
+ Expanding our stock universe to all stocks that trade on Nasdaq.
+ Adding more quality checks and robust handling of issues - what if requests no longer works? What if the Most Active Stocks page disappears?
+ Turning this code into Python scripts, placing them in an Amazon EC2 server, and automating this collection process with cron.

I hope this tutorial was useful to you. Thanks for reading!